<a href="https://colab.research.google.com/github/ngupta23/medium_articles/blob/main/time_series/pycaret/pycaret_ts_multi_step_forecasts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
  import pycaret
except:
  !pip install pycaret-ts-alpha

In [2]:
import numpy as np
from pycaret.datasets import get_data
from pycaret.internal.pycaret_experiment import TimeSeriesExperiment

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


## Get Data

In [3]:
y = get_data("airline")

Period
1949-01    112.0
1949-02    118.0
1949-03    132.0
1949-04    129.0
1949-05    121.0
Freq: M, Name: Number of airline passengers, dtype: float64

## Setup experiment with a forecast horizon

In [4]:
# fh = 3rd year from the end of the data
# 3rd year is the 25th month to 36th month in the future (note numpy is left inclusive and right exclusive)

exp = TimeSeriesExperiment()
exp.setup(data=y, fh=np.arange(25,37), session_id=42)

,Description,Value
0,session_id,42
1,Original Data,"(144, 1)"
2,Missing Values,False
3,Transformed Train Set,"(108,)"
4,Transformed Test Set,"(12,)"
5,Fold Generator,ExpandingWindowSplitter
6,Fold Number,3
7,Enforce Prediction Interval,False
8,Seasonal Period Tested,12
9,Seasonality Detected,True


## Check correctness of setup

In [5]:
#### Check Train-Test Split ----
exp.plot_model(plot="train_test_split")

#### Check cross-validation setup  ----
exp.plot_model(plot="cv")

## Create initial model

In [6]:
model = exp.create_model("exp_smooth")

,cutoff,MAE,RMSE,MAPE,SMAPE,MASE,R2
0,1952-12,17.4879,20.3904,0.0591,0.0605,0.7459,0.7446
1,1953-12,46.3840,49.7991,0.1376,0.1486,1.8868,-0.1810
2,1954-12,59.5921,63.4842,0.1574,0.1717,2.5891,-0.3119
Mean,NaN,41.1547,44.5579,0.1180,0.1269,1.7406,0.0839
SD,NaN,17.5822,17.9791,0.0424,0.0479,0.7595,0.4702


In [7]:
exp.predict_model(model)

,Model,MAE,RMSE,MAPE,SMAPE,MASE,R2
0,Exponential Smoothing,34.5844,38.4441,0.0698,0.0723,1.1312,0.7332


1960-01    393.5358
1960-02    384.0226
1960-03    444.8450
1960-04    432.5152
1960-05    433.9544
1960-06    497.7995
1960-07    551.3423
1960-08    544.9049
1960-09    478.4471
1960-10    415.8106
1960-11    363.6572
1960-12    409.8425
Freq: M, Name: Number of airline passengers, dtype: float64

In [8]:
exp.plot_model(model)

## Finalize model for future forecasts

In [9]:
final_model = exp.finalize_model(model)

In [10]:
exp.predict_model(final_model)

1963-01    502.8965
1963-02    472.0903
1963-03    524.9193
1963-04    558.0197
1963-05    569.5458
1963-06    645.5994
1963-07    746.8286
1963-08    736.6189
1963-09    614.6290
1963-10    548.7643
1963-11    466.7981
1963-12    516.6501
Freq: M, Name: Number of airline passengers, dtype: float64

In [11]:
exp.plot_model(final_model)